In [1]:
import torch

# 1.4.1 텐서

In [5]:
scalar1 = torch.tensor([1.])
print(scalar1)
scalar2 = torch.tensor([3.])
print(scalar2)

tensor([1.])
tensor([3.])


In [9]:
"""
Scalar
"""

# scalar 값의 사칙 연산도 가능(+, -, *, /)
add_scalar = scalar1 + scalar2
print(add_scalar)

tensor([4.])


In [10]:
"""
Vector
"""

vector1 = torch.tensor([1., 2., 3.])
vector2 = torch.tensor([4., 5., 6.])
print(vector1)
print(vector2)

tensor([1., 2., 3.])
tensor([4., 5., 6.])


In [11]:
# 마찬가지로 사칙연산이 가능하지만, 각 요소별로 계산이 진행된다!(element-wise)
add_vector = vector1 + vector2
sub_vector = vector1 - vector2
mul_vector = vector1 * vector2
div_vector = vector1 / vector2
print(add_vector)
print(sub_vector)
print(mul_vector)
print(div_vector)

tensor([5., 7., 9.])
tensor([-3., -3., -3.])
tensor([ 4., 10., 18.])
tensor([0.2500, 0.4000, 0.5000])


In [12]:
# torch 내장연산도 가능
torch.add(vector1, vector2)
torch.dot(vector1, vector2) # 내적

tensor(32.)

In [13]:
"""
행렬(Matrix)
"""

matrix1 = torch.tensor([[1., 2.], [3., 4.]])
print(matrix1)

tensor([[1., 2.],
        [3., 4.]])


In [14]:
matrix2 = torch.tensor([[5., 6.], [7., 8.]])
print(matrix2)

tensor([[5., 6.],
        [7., 8.]])


In [15]:
# 사칙 연산도 벡터와 동일한 방법으로 가능
torch.add(matrix1, matrix2)

tensor([[ 6.,  8.],
        [10., 12.]])

In [17]:
# 행렬 곱셈 연산은 각 요소별로 계산
# 이에 비해 행렬 곱 연산은 torch.matmul을 이용해 실행할 수 있으며 우리가 아는 그 행렬을 곱하는 방법으로 계산된다.
print(torch.mul(matrix1, matrix2))
print(torch.matmul(matrix1, matrix2))

tensor([[ 5., 12.],
        [21., 32.]])
tensor([[19., 22.],
        [43., 50.]])


In [19]:
"""
텐서(Tensor)
"""
# 행렬을 2차원 배열이라 표현할 수 있다면, 텐서는 2차원 이상의 배열이라 할 수 있다.
tensor1 = torch.tensor([[[1., 2.], [3., 4.]], [[5., 6.], [7., 8.]]])
print(tensor1)
tensor2 = torch.tensor([[[9., 10.], [11., 12.]], [[13., 14.], [15., 16.]]])
print(tensor2)

tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])
tensor([[[ 9., 10.],
         [11., 12.]],

        [[13., 14.],
         [15., 16.]]])


In [20]:
print(torch.mul(tensor1, tensor2))
print(torch.matmul(tensor1, tensor2))

tensor([[[  9.,  20.],
         [ 33.,  48.]],

        [[ 65.,  84.],
         [105., 128.]]])
tensor([[[ 31.,  34.],
         [ 71.,  78.]],

        [[155., 166.],
         [211., 226.]]])


# 1.4.2 Autograd
- 파이토치는 Autograd 방식으로 역전파를 이용해 파라미터를 업데이트 하는 방법을 쉽게 구현할 수 있도록 설정되어 있음.
- 간단한 딥러닝 모델을 설계하고, 파라미터를 어떻게 업데이트 할 수 있는지를 알아보자!

In [21]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

In [22]:
print(DEVICE)

cpu


In [23]:
# 딥러닝 모델에서 파라미터 업데이트 시 계산되는 데이터의 개수
# 배치 수만큼 데이터 이용해 output을 계산하고, 배치 수만큼 출력된 결괏값에 대한 오차값 계산
# 배치 수만큼 계산된 오차값을 평균 해 역전파를 적용하고 이를 바탕으로 파라미터 업데이트
BATCH_SIZE = 64

# input의 크기이자 입력층 노드 수, 즉 입력 벡터 값을 의미한다!
INPUT_SIZE = 1000

# input을 파라미터를 이용해 계산한 결과에 한번 더 계산되는 파라미터 수 의미
# 즉 입력층에서 은닉층으로 전달되었을 때, 은닉층의 노드 수
# 예제) input = (64, 1000), (1000, 100) 크기의 행렬과 행렬 곱을 계산하기 위해 설정한 수
HIDDEN_SIZE = 100

# 최종으로 출력되는 값의 벡터 크기
OUTPUT_SIZE = 10

In [26]:
"""
데이터와 파라미터 설정
"""
# randn: 평균이 0, 표준편차가 1인 정규분포에서 샘플링한 값
# 즉, 아래 식을 사용해서 크기가 1,000짜리 벡터를 64개 만들 수 있으며, x는 (64, 1000) 모양의 데이터가 된다.
# 그리고 x, y는 input, output이기 때문에 gradient를 계산할 필요가 없기 때문에 requires_grad는 False가 된다.
x = torch.randn(BATCH_SIZE,
                 INPUT_SIZE,
                 device = DEVICE,
                 dtype = torch.float,
                 requires_grad = False)
y = torch.randn(BATCH_SIZE,
               OUTPUT_SIZE,
               device = DEVICE,
               dtype = torch.float,
               requires_grad = False)

# w1은 input의 데이터 크기가 1,000이며 이와 행렬 곱을 하기 위해서는 다음 행의 값도 1,000이어야 함
# 또한 행렬 곱을 이용해 100 크기의 데이터를 생성하기 위해 (1000, 100) 크기의 데이터를 생성해야 한다.
w1 = torch.randn(INPUT_SIZE,
                HIDDEN_SIZE, device = DEVICE,
                 dtype = torch.float,
                 requires_grad = True)

# w2는 w1과 x를 행렬 곱 한 결과와 계산할 수 있는 데이터여야 한다. 
# torch.matmul(x, w1) 한 결과는 (64, 100)이며, (100, 10) 행렬을 통해 output을 계산할 수 있도록 w2의 모양을 설정한다.
# 따라서 hidden=100, output=10
w2 = torch.randn(HIDDEN_SIZE,
                OUTPUT_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = True)

In [31]:
learning_rate = 1e-6
for t in range(1, 501):
    # x와 w1 간 행렬 곱을 취해 나온 결과값 계산
    # torch 모듈 내 clamp라는 메서드 이용해 비선형 함수 적용
    """
    * 딥러닝 모델에서는 층과 층 사이에 비선형 함수를 이용해 높은 표현력을 가지는 방정식을 얻게 됨
    * 여기서 clamp는 비선형 함수 ReLU()와 같은 역할을 수행하게 된다: 최솟값 0, 0보다 큰 값은 자기 자신을 가지는 메서드
    """
    # clamp를 이용해 계산된 결과와 w2를 이용해 행렬 곱을 한번 더 계산
    y_pred = x.mm(w1).clamp(min = 0).mm(w2)
    
    # pow: 지수를 취하는 메서드
    # loss => 차의 제곱의 합이 된다.
    loss = (y_pred - y).pow(2).sum()
    
    if t % 100 == 0:
        print("Iteration: ", t, "\t", "Loss: ", loss.item())
        print("after bp: ", w1, "\t", w2)
        
    # backward(): 각 파라미터 값에 대해 Gradient를 계산하고 이를 통해 Back Propagation을 진행
    loss.backward()
    
    #  파라미터 값에 대해 gradient를 계산한 결과를 이용해 파라미터 값을 업데이터 할 때, 해당 시점의 gradient 값을 고정한 후 업데이트를 진행한다.
    with torch.no_grad():
        # w1.grad: w1의 Gradient 값을 의미
        # 빼주는 이유는 Loss값이 최소로 계산될 수 있는 파라미터를 찾기 위함
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        if t % 100 == 0:
            print("after update: ", w1, "\t", w2)
        
        # 각 파라미터 값의 gradient를 초기화 해 다음 반복문을 진행할 수 있도록 0으로 설정
        w1.grad.zero_()
        w2.grad.zero_()

Iteration:  100 	 Loss:  1.4966524304327322e-06
after bp:  tensor([[ 1.3588,  1.0412,  0.5853,  ..., -1.2567,  0.0510, -0.5038],
        [ 1.3045,  1.9159, -0.4097,  ...,  0.3164,  0.5295,  1.1521],
        [-0.7513, -0.5107, -0.9550,  ...,  2.0328, -2.1945, -0.6990],
        ...,
        [-1.3465,  0.0377, -1.8642,  ..., -0.3001,  0.8504, -0.8988],
        [ 0.1379,  1.3396, -0.7972,  ...,  1.0240, -0.4708,  0.9498],
        [-0.9051,  0.9616, -1.3023,  ..., -0.5995,  0.1714, -0.5080]],
       requires_grad=True) 	 tensor([[ 1.5143e+00,  2.8529e-03,  1.2348e+00,  1.3174e+00, -3.6637e-01,
          2.9446e-01, -5.1002e-01, -2.7001e-01,  3.3761e-01,  3.8025e-01],
        [ 3.7998e-01,  8.6241e-01,  1.4347e+00,  1.4627e-03,  7.4442e-01,
          1.1621e+00, -1.0427e+00, -7.3224e-01,  1.1897e+00, -2.1270e-01],
        [ 3.1268e-01,  5.7659e-01, -2.2673e-01, -7.1786e-01,  1.0232e+00,
          4.9743e-01,  1.4302e+00, -1.1368e-01,  1.1211e+00, -3.3624e-01],
        [ 3.8310e-01, -9.7240e-